In [12]:
import scanpy as sc
import os
import random
import pickle
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
data_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/pbmc"

In [3]:
adata = sc.read_h5ad("/lustre/groups/ml01/workspace/ot_perturbation/data/pbmc/adata_hvg2000_LV.h5ad")

In [4]:
with open("/lustre/groups/ml01/workspace/ot_perturbation/data/pbmc/cyto_order.pkl", "rb") as file:
    cyto_order = pickle.load(file)

In [5]:
adata.X = adata.layers["counts"]
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [6]:
unique_donors = adata.obs["donor"].unique()
unique_cytokines = list(set(adata.obs["cytokine"].unique()) - set(["PBS"]))

In [7]:
donor_control_vecs = {}
for donor in unique_donors:
    donor_control_vecs[donor] = adata[(adata.obs["donor"]==donor) & (adata.obs["cytokine"]=="PBS")].X.mean(axis=0)

In [13]:
donor_cytokine_displacements = {}
for donor in unique_donors:
    donor_cytokine_displacements[donor] = {}
    for cytokine in unique_cytokines:
        donor_cytokine_displacements[donor][cytokine] = np.asarray(adata[(adata.obs["donor"]==donor) & (adata.obs["cytokine"]==cytokine)].X.mean(axis=0) -donor_control_vecs[donor])
    

In [16]:
import json
with open("/lustre/groups/ml01/workspace/ot_perturbation/data/pbmc/donor_cytokine_displacements.pkl", "wb") as file:
    pickle.dump(donor_cytokine_displacements, file)